<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/warp_bubble_pinn_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch numpy matplotlib

In [ ]:
#!/usr/bin/env python3
"""
warp_bubble_pinn.py

End‐to‐end physics‐informed WarpBubbleAI pipeline:
  1. Synthetic dataset: (energy_density, curvature) → (bubble_radius, thickness)
  2. PINN loss: data MSE + physics residual enforcing volume‐to‐energy ratio
  3. MLP with Dropout & LayerNorm for uncertainty
  4. MC‐Dropout inference for predictive uncertainty
  5. Training loop with AdamW, ReduceLROnPlateau, early stopping
  6. Evaluation: scatter plots, training curves, and uncertainty heatmap
"""

import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# ------------------------------------------------------------------------------
# 1. Synthetic Data Generator
# ------------------------------------------------------------------------------

def true_warp_params(energy, curvature):
    """
    Toy mapping:
      bubble_radius = (energy * curvature)**0.4
      thickness     = log(1 + energy/curvature)
    """
    R = (energy * curvature)**0.4
    T = np.log1p(energy/curvature)
    return R, T

class SyntheticWarpBubbleDataset(Dataset):
    def __init__(self, n_samples=10000, seed=0):
        np.random.seed(seed)
        # energy_density ∈ [0.1, 10], curvature ∈ [0.1, 10]
        E = np.random.uniform(0.1, 10.0, size=(n_samples,)).astype(np.float32)
        C = np.random.uniform(0.1, 10.0, size=(n_samples,)).astype(np.float32)
        R, T = true_warp_params(E, C)
        Y = np.stack([R, T], axis=1).astype(np.float32)

        # Normalize inputs and outputs
        self.X = torch.from_numpy(
            np.stack([
                (E - E.mean())/E.std(),
                (C - C.mean())/C.std()
            ], axis=1)
        )
        self.Y = torch.from_numpy(
            np.stack([
                (R - R.mean())/R.std(),
                (T - T.mean())/T.std()
            ], axis=1)
        )
        # Store normalization stats for later inverse-transform
        self.stats = {
            'E_mean': E.mean(), 'E_std': E.std(),
            'C_mean': C.mean(), 'C_std': C.std(),
            'R_mean': R.mean(), 'R_std': R.std(),
            'T_mean': T.mean(), 'T_std': T.std()
        }

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

# ------------------------------------------------------------------------------
# 2. Model Definition: WarpBubbleAI
# ------------------------------------------------------------------------------

class WarpBubbleAI(nn.Module):
    def __init__(self, input_dim=2, hidden_dims=[64,64], output_dim=2, p_drop=0.1):
        super().__init__()
        layers = []
        dim = input_dim
        for h in hidden_dims:
            layers += [
                nn.Linear(dim, h),
                nn.LayerNorm(h),
                nn.ReLU(),
                nn.Dropout(p_drop)
            ]
            dim = h
        layers.append(nn.Linear(dim, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# ------------------------------------------------------------------------------
# 3. Physics‐Informed Loss
# ------------------------------------------------------------------------------

def physics_residual(pred, inputs, stats):
    """
    Enforce bubble volume ∝ energy/curvature:
      V = 4/3 π R^3
      target_ratio = energy / curvature
      residual = (V_pred - k*target_ratio)^2
    We absorb constants into λ_phys.
    """
    # Un-normalize R and T
    R = pred[:,0] * stats['R_std'] + stats['R_mean']
    # compute volume
    V_pred = (4/3) * np.pi * R.pow(3)
    # recover energy and curvature
    E = inputs[:,0] * stats['E_std'] + stats['E_mean']
    C = inputs[:,1] * stats['C_std'] + stats['C_mean']
    ratio = E / C
    # physics loss
    return nn.MSELoss()(V_pred, ratio)

def total_loss(pred, true, inputs, stats, λ_phys=1.0):
    mse = nn.MSELoss()(pred, true)
    phys = physics_residual(pred, inputs, stats)
    return mse + λ_phys*phys, mse, phys

# ------------------------------------------------------------------------------
# 4. MC‐Dropout Inference
# ------------------------------------------------------------------------------

def mc_dropout_predict(model, x, n_samples=50):
    model.train()  # keep dropout active
    preds = []
    with torch.no_grad():
        for _ in range(n_samples):
            preds.append(model(x).cpu().numpy())
    arr = np.stack(preds, axis=0)
    mean = arr.mean(axis=0)
    std  = arr.std(axis=0)
    model.eval()
    return mean, std

# ------------------------------------------------------------------------------
# 5. Training Loop
# ------------------------------------------------------------------------------

def train_model(model, train_loader, val_loader, stats,
                lr=1e-3, weight_decay=1e-5, λ_phys=1.0,
                max_epochs=200, patience=20, device='cpu'):

    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5
    )

    best_val = float('inf')
    wait = 0
    history = {'train_loss': [], 'val_loss': []}

    for epoch in range(1, max_epochs+1):
        # Training
        model.train()
        train_loss = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss, mse_l, phys_l = total_loss(pred, yb, xb, stats, λ_phys)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * xb.size(0)
        train_loss /= len(train_loader.dataset)

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                pred = model(xb)
                loss, mse_l, phys_l = total_loss(pred, yb, xb, stats, λ_phys)
                val_loss += loss.item() * xb.size(0)
        val_loss /= len(val_loader.dataset)

        scheduler.step(val_loss)
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)

        print(f"Epoch {epoch:03d} | Train {train_loss:.4e} | Val {val_loss:.4e}")

        # Early stopping
        if val_loss < best_val:
            best_val = val_loss
            torch.save(model.state_dict(), "best_warpbubble_ai.pth")
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                print(f"Early stopping at epoch {epoch}")
                break

    # Load best weights
    model.load_state_dict(torch.load("best_warpbubble_ai.pth"))
    return history

# ------------------------------------------------------------------------------
# 6. Evaluation & Plots
# ------------------------------------------------------------------------------

def plot_training(history):
    plt.figure()
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'],   label='Val Loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss')
    plt.legend(); plt.title('Training Curve')
    plt.tight_layout(); plt.show()

def plot_scatter(true, pred, name):
    plt.figure()
    plt.scatter(true, pred, s=5, alpha=0.3)
    m, M = true.min(), true.max()
    plt.plot([m,M],[m,M],'r--')
    plt.xlabel(f"True {name}"); plt.ylabel(f"Pred {name}")
    plt.title(f"{name}: True vs Pred")
    plt.tight_layout(); plt.show()

def plot_uncertainty_heatmap(model, stats, device):
    # grid over energy and curvature
    E_vals = np.linspace(0.1, 10.0, 100)
    C_vals = np.linspace(0.1, 10.0, 100)
    EE, CC = np.meshgrid(E_vals, C_vals)
    X = np.stack([(EE - stats['E_mean'])/stats['E_std'],
                  (CC - stats['C_mean'])/stats['C_std']], axis=-1)
    X_tensor = torch.from_numpy(X.reshape(-1,2)).float().to(device)
    mean, std = mc_dropout_predict(model, X_tensor, n_samples=100)
    R_std = std[:,0].reshape(CC.shape)

    plt.figure(figsize=(6,5))
    plt.pcolormesh(EE, CC, R_std, shading='auto', cmap='magma')
    plt.colorbar(label="Std of R_pred")
    plt.xlabel("Energy"); plt.ylabel("Curvature")
    plt.title("Uncertainty Heatmap for Bubble Radius")
    plt.tight_layout(); plt.show()

# ------------------------------------------------------------------------------
# 7. Main Execution
# ------------------------------------------------------------------------------

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Prepare data
    ds = SyntheticWarpBubbleDataset(n_samples=12000)
    stats = ds.stats
    n_val = int(0.2 * len(ds))
    n_trn = len(ds) - n_val
    trn_ds, val_ds = random_split(ds, [n_trn, n_val])
    trn_ld = DataLoader(trn_ds, batch_size=128, shuffle=True)
    val_ld = DataLoader(val_ds, batch_size=256)

    # Build model
    model = WarpBubbleAI(input_dim=2).to(device)

    # Train
    history = train_model(
        model, trn_ld, val_ld, stats,
        lr=1e-3, weight_decay=1e-5, λ_phys=1.0,
        max_epochs=200, patience=20, device=device
    )

    # Plots
    plot_training(history)

    # Scatter true vs pred
    # Collect full dataset predictions
    X_all = ds.X.to(device)
    Y_all = ds.Y.numpy()
    with torch.no_grad():
        Y_pred = model(X_all).cpu().numpy()
    # inverse‐transform
    R_true = Y_all[:,0] * stats['R_std'] + stats['R_mean']
    T_true = Y_all[:,1] * stats['T_std'] + stats['T_mean']
    R_pred = Y_pred[:,0] * stats['R_std'] + stats['R_mean']
    T_pred = Y_pred[:,1] * stats['T_std'] + stats['T_mean']

    plot_scatter(R_true, R_pred, "Bubble Radius")
    plot_scatter(T_true, T_pred, "Thickness")

    # Uncertainty heatmap
    plot_uncertainty_heatmap(model, stats, device)